<a href="https://colab.research.google.com/github/mohansharma077/MuRIL-Multilingual-Representation-for-Indian-Langauge-/blob/main/MuRIL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install bert_for-tf2
!pip install tensorflow-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for bert_for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30531 sha256=36cc90bdb4d6886687df02b0624fac2a52c24ed10d8234355b53e8cdca4adb7c
  Stored in directory: /root/.cache/pip/wheels/6f/c7/91/f2b2c2b3cec30578c5de7c27ac99659a2013501dd66e7e3db0
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19471 sha256=fe034463f3f8e1d27041041f5b9f959ec3763a7671fc1c2539ea05902c56978b
  Stored in directory: /root/.cache/pip/wheels/be/17/6c/5c924411a614ee0a74b2dc4f04c9e61dacc4e60fe9854f4f70
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7910 sha256=dee4e459628b68d25579a24e6150bb73a723f1e9ed3fdde4bc5715cb4e35284b
  Sto

In [4]:
#importing necessary library
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import tensorflow_text as text
from bert import bert_tokenization
import numpy as np
from scipy.spatial import distance

In [6]:
def get_model(model_url,max_seq_length):
  inputs=dict(
      input_word_ids=tf.keras.layers.Input(shape=(max_seq_length,),dtype=tf.int32),
      input_mask=tf.keras.layers.Input(shape=(max_seq_length,),dtype=tf.int32),
      input_type_ids=tf.keras.layers.Input(shape=(max_seq_length,),dtype=tf.int32),
  )
  muril_layer=hub.KerasLayer(model_url,trainable=True)
  outputs=muril_layer(inputs)
  assert 'sequence_output' in outputs
  assert 'pooled_output' in outputs
  assert 'encoder_outputs' in outputs
  assert 'default' in outputs
  return tf.keras.Model(inputs=inputs,outputs=outputs["pooled_output"]), muril_layer


In [9]:
max_seq_length=128
muril_model,muril_layer=get_model(model_url="https://tfhub.dev/google/MuRIL/1",max_seq_length=max_seq_length)



In [10]:
vocab_file=muril_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case=muril_layer.resolved_object.do_lower_case.numpy()
tokenizer=bert_tokenization.FullTokenizer(vocab_file, do_lower_case)


In [11]:
def create_input(input_strings, tokenizer, max_seq_length):
  input_ids_all, input_mask_all, input_type_ids_all = [], [], []
  for input_string in input_strings:
    input_tokens = ["[CLS]"] + tokenizer.tokenize(input_string) + ["[SEP]"]
    input_ids = tokenizer.convert_tokens_to_ids(input_tokens)
    sequence_length = min(len(input_ids), max_seq_length)
    
    if len(input_ids) >= max_seq_length:
      input_ids = input_ids[:max_seq_length]
    else:
      input_ids = input_ids + [0] * (max_seq_length - len(input_ids))

    input_mask = [1] * sequence_length + [0] * (max_seq_length - sequence_length)

    input_ids_all.append(input_ids)
    input_mask_all.append(input_mask)
    input_type_ids_all.append([0] * max_seq_length)
  
  return np.array(input_ids_all), np.array(input_mask_all), np.array(input_type_ids_all)
     

In [12]:
def encode(input_text):
  input_ids,input_mask,input_type_ids=create_input(input_text,
                                                   tokenizer,
                                                   max_seq_length)
  inputs=dict(
      input_word_ids=input_ids,
      input_mask=input_mask,
      input_type_ids=input_type_ids,
  )
  return muril_model(inputs)


In [13]:

sentences = ["दोस्त", "मित्र", "शत्रु"]

In [14]:

embeddings = encode(sentences)

In [15]:

dst_1 = distance.euclidean(np.array(embeddings[0]), 
                           np.array(embeddings[1]))
print("Distance between {} & {} is {}".format(sentences[0],
                                                sentences[1],
                                                dst_1))

Distance between दोस्त & मित्र is 0.009007949382066727


In [16]:


dst_2 = distance.euclidean(np.array(embeddings[1]), 
                           np.array(embeddings[2]))
print("Distance between {} & {} is {}".format(sentences[1],
                                                sentences[2],
                                                dst_2))
     


Distance between मित्र & शत्रु is 0.011569392867386341
